In [2]:
import sys
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
import functions
import functions2
from pprint import pprint
import json
from scipy import interpolate
import langmuir.gymness.aggregate as agg
import pandas as pd
from pandas.io.json import json_normalize
%matplotlib

Using matplotlib backend: GTK3Agg


In [3]:
with open('../config_2702.json') as file:
    data = json.load(file)
    
df = pd.DataFrame()
for item, content in data['T'].items():
    df_item = pd.read_json(json.dumps(content))
    df_item['file_name'] = '../data/2702/2702spectr/{}.dat'.format(item)
    df=df.append(df_item)

In [4]:
def get_i_abs(x):
    [i, l] = functions2.get_intensity(x['file_name'], x['l'], x['t'])              
    tau = functions2.get_calibration_coefficient(l)
    return functions2.get_absolute_intensity(i, l, tau, x['l'], )

def get_tau(x):
    i, l = np.genfromtxt(x['file_name'], unpack=True)
    return functions2.get_calibration_coefficient(l)

a = df.apply(get_i_abs, axis=1)
df['i_abs'] = a[0]
df['correct_l'] = a[1]
# df['tau'] = df.apply(get_tau, axis=1)

In [5]:
df_merged = pd.merge(df, df, on=['w', 'p_off'])
df_merged_hahb = df_merged[(df_merged.l_x == 656.148) & (df_merged.l_y == 486)].reset_index()
df_merged_hahb['ratio'] = df_merged_hahb['i_abs_x'] / df_merged_hahb['i_abs_y']
# df_merged_hahb = df_merged_hahb.groupby(['p_off', 'w']).mean()
df_merged_hahb['T_lavrov'] = functions2.get_temperature(df_merged_hahb['ratio'], '../data/2702/ratio_de(Lavrov).csv')
df_merged_hahb['T'] = functions2.get_temperature(df_merged_hahb['ratio'], '../data/2702/ratio_de_hahb.csv')
# df_merged_hahb

In [6]:
df_merged_hahd = df_merged[(df_merged.l_x == 656.148) & (df_merged.l_y == 410.082)].reset_index()
df_merged_hahd['ratio'] = df_merged_hahd['i_abs_x'] / df_merged_hahd['i_abs_y']
# df_merged_hahd = df_merged_hahd.groupby(['p_off', 'w']).mean()
df_merged_hahd['T'] = functions2.get_temperature(df_merged_hahd['ratio'], '../data/2702/ratio_de_hahd.csv')
df_merged_hahd = df_merged_hahd.dropna().groupby(['p_off', 'w']).mean()
df_merged_hahd

index      l_x       p_x   t_x        i_abs_x  correct_l_x  \
p_off    w                                                                     
0.000493 20  529.166667  656.148  0.000498  0.05  279540.000000      656.148   
         30  738.000000  656.148  0.000532  0.05  705873.333333      656.148   
         40  652.333333  656.148  0.000513  0.05  625240.000000      656.148   
         50  952.000000  656.148  0.000530  0.05  743046.666667      656.148   
         60   99.666667  656.148  0.000513  0.05  579726.666667      656.148   

                 l_y       p_y  t_y       i_abs_y  correct_l_y      ratio  \
p_off    w                                                                  
0.000493 20  410.082  0.000498  0.2  11751.666667   410.090000  23.793721   
         30  410.082  0.000532  0.2  27380.000000   410.090000  25.782241   
         40  410.082  0.000513  0.2  23780.000000   410.082000  26.294015   
         50  410.082  0.000530  0.2  29291.666667   410.090000  25.367322   
         60  410.082  0.000513  0.2  23293.333333   410.089333  24.889893   

                    T  
p_off    w             
0.000493 20  8.345771  
         30  4.171503  
         40  3.221569  
         50  5.000909  
         60  5.865339

In [10]:
# t1, r1 = np.genfromtxt('../data/2702/ratio_de_hahd.csv', unpack=True)
t2, r2 = np.genfromtxt('../data/2702/ratio_de_hahb.csv', unpack=True)
tl, rl = np.genfromtxt('../data/2702/ratio_de(Lavrov).csv', unpack=True)
plt.subplot(211)
plt.title('$H_{\\alpha} / H_{\\delta}$')
plt.plot(tl, rl, label='Adas database')
plt.xlabel('$T_e$', fontsize=14)
plt.ylabel('ratio', fontsize=14)
plt.legend()
plt.grid()

plt.subplot(212)
plt.title('$H_{\\alpha}/H_{\\beta}$')
plt.plot(tl, rl, label='Lavrov database')
plt.plot(t2, r2, label='Adas database')
plt.xlabel('$T_e$', fontsize=14)
plt.ylabel('ratio', fontsize=14)
plt.legend()
plt.grid()
plt.tight_layout()

In [11]:
# langmuir density and profile temperature
df_langmuir = agg.aggregate_data('../langmuir/data/data-2702')
# the grouped by columns will become indexes
p_grouped_langmuir = df_langmuir.groupby(['P_off', 'W_em'])['n_e', 'err_n_e', 'T_e', 'err_T_e'].mean()
# set plot stles, titles and figures. Need to add the labels though

Reading dataset:['2702lang10', '2702lang07', '2702lang12', '2702lang02', '2702lang03', '2702lang08', '2702lang04', '2702lang13', '2702lang09', '2702lang05', '2702lang18', '2702lang17', '2702lang19', '2702lang14', '2702lang15', '2702lang06', '2702lang11', '2702lang16', '2702lang01']


In [12]:
# plotting data
plt.subplot(221)
plt.title('p = {:.2e} mbar'.format(df_langmuir['p_off (mbar)'][0]), fontsize=20)
plt.errorbar(df['w (%)'][:3], df['n_e'][:3], yerr=df['err_n_e'][:3], fmt='o', label='Langmuir')
plt.errorbar(df_spectr['w (%)'][:3], df_spectr['n ($cm^{-3}$)'][:3], yerr=(df_spectr['n_max ($cm^{-3}$)'] - df_spectr['n_min ($cm^{-3}$)'])[:3], 
             fmt='o', label='OES')
plt.grid()
plt.xlabel('W (%)', fontsize=18)
plt.ylabel('$n_e$ $cm^{-3}$', fontsize=18)
plt.legend()
'''
plt.subplot(223)
plt.errorbar(df['w (%)'][:3], df['T_e'][:3], yerr=df['err_T_e'][:3], fmt='o', label='Langmuir')
plt.plot(df_spectr['w (%)'][:3], df_spectr['T'][:3], 'o', label='OES')
plt.grid()
plt.xlabel('W (%)', fontsize=18)
plt.ylabel('$T_e$ (eV)', fontsize=18)
plt.legend()
plt.tight_layout()

plt.subplot(222)
plt.title('p = {:.2e} mbar'.format(df['p_off (mbar)'][4]), fontsize=20)
plt.errorbar(df['w (%)'][3:], df['n_e'][3:], yerr=df['err_n_e'][3:], fmt='o', label='Langmuir')
plt.errorbar(df_spectr['w (%)'][3:], df_spectr['n ($cm^{-3}$)'][3:], yerr=(df_spectr['n_max ($cm^{-3}$)'] - df_spectr['n_min ($cm^{-3}$)'])[3:], 
             fmt='o', label='OES')
plt.grid()
plt.xlabel('W (%)', fontsize=18)
plt.ylabel('$n_e$ $cm^{-3}$', fontsize=18)
plt.legend()

plt.subplot(224)
plt.errorbar(df['w (%)'][:3], df['T_e'][3:], yerr=df['err_T_e'][3:], fmt='o', label='Langmuir')
plt.plot(df_spectr['w (%)'][3:], df_spectr['T'][3:], 'o', label='OES')
plt.grid()
plt.xlabel('W (%)', fontsize=18)
plt.ylabel('$T_e$ (eV)', fontsize=18)
plt.legend()
plt.tight_layout()
'''

KeyError: 'p_off (mbar)'

In [92]:
# langmuir density and profile temperature
#                        path to results data
df_lang = agg.aggregate_data('../langmuir/data/data-2702')
# the grouped by columns will become indexes
a = df_lang.groupby(['P_off', 'W_em'])['n_e', 'err_n_e', 'T_e', 'err_T_e'].mean()
# set plot stles, titles and figures. Need to add the labels though
pressures = a.index.get_level_values(0).unique()
i = 0
j = 2
for pressure in pressures:
    plt.subplot(221 + i)
    plt.grid()
    plt.title('Pressure: {:.2e} mbar'.format(pressure))
    w = a.xs(pressure).index.values
    try:
        T_e_spectr = df_merged_hahd.xs(round(pressure, 6))['T']
        plt.plot(w, T_e_spectr, '-o', label='OES $H_{\\alpha}/H_{\\delta}$')
    except: 
        pass
    T_e_lang = a.xs(pressure)['T_e']
    err_T_e_lang = a.xs(pressure)['err_T_e']
    plt.errorbar(w, T_e_lang, yerr=err_T_e_lang, label='Langmuir', fmt='-o')
    plt.xlabel('W (%)')
    plt.ylabel('$T_e$ (eV)')  
    plt.legend()
    
    plt.subplot(221 + j)
    n_e = a.xs(pressure)['n_e'] / 1e6
    err_n_e = a.xs(pressure)['err_n_e'] / 1e6
    plt.errorbar(w, n_e, yerr=err_n_e, label='Langmuir', fmt='-o')
    plt.xlabel('W (%)', fontsize=13)
    plt.ylabel('$n_e (cm^{-3})$', fontsize=13)
    plt.grid()
    plt.legend()
    plt.tight_layout()
    i = i + 1
    j = j + 1 

Reading dataset:['2702lang16', '2702lang14', '2702lang19', '2702lang02', '2702lang17', '2702lang04', '2702lang08', '2702lang15', '2702lang03', '2702lang09', '2702lang07', '2702lang12', '2702lang06', '2702lang01', '2702lang13', '2702lang18', '2702lang10', '2702lang11', '2702lang05']


In [84]:
df_merged_hahd.xs(round(pressures[1],6))['T']


w
20    8.345771
30    4.171503
40    3.221569
50    5.000909
60    5.865339
Name: T, dtype: float64